## Preparing a Dataset of Job Postings in the Data Field

The goal of this notebook is to prepare the job postings dataset for insertion into a MySQL database. The database has been specifically designed to store this data in a normalized structure.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('data_jobs_salary_all.xlsx')

In [3]:
df.head(1).T

,0
job_title_short,Senior Data Scientist
job_title,Senior Data Scientist Data and Analytics Perfo...
job_location,"Bennington, NE"
job_via,via ZipRecruiter
job_schedule_type,Full-time
job_work_from_home,False
search_location,Sudan
job_posted_date,2023-04-24 09:51:15
job_no_degree_mention,False
job_health_insurance,True


In [4]:
df.shape

(32672, 16)

In [5]:
df.isna().sum()

job_title_short              0
job_title                    0
job_location               355
job_via                     10
job_schedule_type          141
job_work_from_home           0
search_location              0
job_posted_date              0
job_no_degree_mention        0
job_health_insurance         0
job_country                  0
salary_rate                  0
salary_year_avg          10636
salary_hour_avg          22036
company_name                 0
job_skills                3187
dtype: int64

## Extracting Unique Values for Dimension Tables
This section focuses on extracting unique values from several columns to create dimension tables for data normalization. The unique values for each dimension will be saved to a separate CSV file. These CSV files will then be loaded into their corresponding tables in the database.
 
The general procedure for each column is as follows:
1. Extract the unique values from the column.
2. Clean these unique values if necessary.
3. Create a pandas Series from the unique values.
4. Export the Series to a CSV file.

### Unique 'job_title_short'

In [6]:
job_titles = ( pd.Series(df.job_title_short.unique(), name = 'job_title')
              .sort_values()
              .reset_index(drop = True)
)

The `AUTO_INCREMENT` values for the primary keys in the SQL tables start from 1. However, the default index for a pandas Series starts from 0. To ensure the foreign keys match correctly when we map them later, we will adjust the Series index to start from 1 instead of 0. 

Solution: to add 1 to the indexes of the series so they start from 0. 

In [7]:
job_titles.index = job_titles.index + 1

In [8]:
job_titles

1              Business Analyst
2                Cloud Engineer
3                  Data Analyst
4                 Data Engineer
5                Data Scientist
6     Machine Learning Engineer
7           Senior Data Analyst
8          Senior Data Engineer
9         Senior Data Scientist
10            Software Engineer
Name: job_title, dtype: object

In [9]:
job_titles.to_csv('./csv/job_titles.csv', index = False)

### Unique 'company_name'

In [10]:
# df.company_name.nunique()

In [11]:
df.company_name = df.company_name.str.replace('\u200b', '').str.strip()

In [12]:
print(*df.company_name.sort_values().unique(), sep = '\n')

#twiceasnice Recruiting
(THE VANGUARD GROUP/MALVERN,PA)
/dev/color
0nward Select
1 POINT SYSTEM LLC
1 Point System
1 Point System LLC.
1 Point system
1 Point system LLC
1 point system
1-800-FLOWERS.COM, INC.
1001 Absa Bank
100Insure
10x Genomics
16 Points Holdings LLC
1840 & Company
1872 Consulting
1904labs
1st Edge
1st United Services Credit Union
1st-Recruit LLC
1stdibs.com
21Tech, LLC
22nd Century Technologies Inc. (TSCTI)
22nd Century Technologies, Inc.
24 Capital, LLC
24 Seven Talent
247hire
24x7 ESI, Inc.
2K
2U
360insights.com
360itservices
3B Staffing LLC
3Core Systems Inc
3Core Systems Inc.
3E
3G Federal Solutions, LLC
3K Technologies
3M
3M Group
3Pillar Global
3Q Digital
3Red Partners
4 Corner Resources
4-Serv Solutions Inc.
500 Global
55 SILVER
605
66degrees
6e Tech
6point6
6sense
710 Labs
81qd
84.51
84.51°
9 Com Technologies, inc
9 Silver LLC
9992
A Safe Haven Foundation
A great organization!
A*TEAM Collision Center
A+ Consulting
A-Line Staffing Solutions
A3 Staffing Solutio

In [13]:
df.loc[df.company_name == '(THE VANGUARD GROUP/MALVERN,PA)', 'company_name'] = 'THE VANGUARD GROUP'.title()
df.company_name = df.company_name.str.replace('#twiceasnice', 'twiceasnice')

In [14]:
df.loc[df.company_name.str.startswith('0'), 'company_name']

15279    0nward Select
16632    0nward Select
Name: company_name, dtype: object

'0nward select' has a zero as initial O

In [15]:
df.loc[df.company_name == '0nward Select', 'company_name'] = 'Onward Select'

*Openstaff* has a weird font and won't be recognized as simple string

In [16]:
df.company_name.sort_values().tail(5)

22652                  zooplus SE
10615                         ztp
3245                        ЛАНИТ
18882    Технологическая компания
1759                    𝐎𝐩𝐞𝐧𝐬𝐭𝐚𝐟𝐟
Name: company_name, dtype: object

In [17]:
df.loc[1759, 'company_name'] = 'Openstaff'

### Cleaning Company Names with AI Assistance

A text file containing all company names is created. An AI assistant will be used to process this file, cleaning and standardizing the names to handle variations.

Get a text file with all the values for 'company_name'

In [18]:
with open('df_companies.txt', 'w') as f:
    for value in df.company_name:
        f.write(f'{value}\n')

Process the file using the script 'clean_companies.py' and then read the output file

In [19]:
import clean_companies as clean

In [20]:
# Process the file
clean_companies = clean.process_company_names('df_companies.txt', 'cleaned_companies.txt')

Original number of companies: 32672
Unique normalized companies: 9258


In [21]:
clean_companies[:10]

['Cox Communications',
 'Worldgate',
 'National Technical Information Service',
 'Bosch Group',
 'Presidio',
 'Lmi Consulting',
 'Multiplan',
 'Sports Info Solutions',
 'Arsenault',
 'TikTok']

Compare the results of company names before and after cleaning

In [22]:
sample_companies = df.company_name.sample(10)
indexes = sample_companies.index
clean_companies_series = pd.Series(clean_companies)

In [23]:
pd.DataFrame({'df' : sample_companies, 'clean' : clean_companies_series[indexes]})

,df,clean
29452,Deezer,Deezer
21632,Electronic Arts,Electronic Arts
10386,Adex Corporation,Adex
8255,Neara,Neara
15356,Algo Capital Group,Algo Capital Group
11394,Care.com,Carecom
12733,Russell Tobin,Russell Tobin
15945,Rangam,Rangam
13344,Salesforce,Salesforce
26115,Community Health Plan Of Washington,Community Health Plan Of Washington


Finally, reassign the clean company names to the dataframe column

In [24]:
df.company_name = clean_companies

Some 'company_name' values are empty (they were Co. Inc., Company... and so on)

In [25]:
df.company_name[df.company_name == '']

1038     
2735     
24027    
25882    
27635    
29441    
31588    
Name: company_name, dtype: object

In [26]:
df.loc[df.company_name == '', 'company_name'] = 'NULL'

Fix some issues with values that are not estrictly duplicated, but will raise an error as duplicates when loading the data into the database.

In [27]:
df.company_name[df.company_name.str.contains('Mondel')]

16953    Mondelēz International
28685    Mondelez International
Name: company_name, dtype: object

In [28]:
df.company_name[df.company_name.str.contains('Nestl')]

838                     Nestlé
2240                    Nestlé
4120                    Nestlé
5843                    Nestlé
8896                    Nestlé
9238                    Nestlé
10826                   Nestlé
13061                   Nestle
15780                   Nestlé
16330                   Nestlé
17430                   Nestle
17688                   Nestlé
17784    Nestl Purina Pet Care
17821                   Nestle
21907                   Nestlé
23068                   Nestle
25319                   Nestlé
25890                   Nestle
30503                   Nestlé
Name: company_name, dtype: object

In [29]:
df.loc[df.company_name == 'Nestle', 'company_name'] = 'Nestlé'

In [30]:
df.loc[df.company_name.str.contains('Mondel'), 'company_name'] = 'Mondelez'

Get list of unique companies:

In [31]:
companies = (pd.Series(df.company_name.
                       drop_duplicates()
                       .sort_values()
                       .reset_index( drop = True)
                       , name = 'company')
)
companies.index += 1

In [32]:
companies

1                 1 Point System
2                 1001 Absa Bank
3                      100Insure
4                   10X Genomics
5             16 Points Holdings
                  ...           
9252      Zurich Insurance Group
9253                       Zynga
9254           Zyreoneconsulting
9255                       Ланит
9256    Технологическая Компания
Name: company, Length: 9256, dtype: object

In [33]:
companies.to_csv('./csv/companies.csv', index = False)

### Unique locations

In [34]:
# df.job_country.unique()

In [35]:
countries = (pd.Series(df.job_country
                       .dropna()
                       .drop_duplicates()
                       .sort_values()
                       .reset_index(drop = True),
                       name = 'country' )
)
countries.index += 1

I will add Georgia to the countries table.

In [36]:
countries = pd.concat([countries, pd.Series('Georgia')])

countries = countries.sort_values().reset_index(drop = True)
countries.index += 1
countries.name = 'country'

In [37]:
# countries.head()

In [38]:
countries.to_csv('./csv/countries.csv', index = False)

#### Unique 'job_location'

In [39]:
# df.job_location.unique()

In [40]:
df.job_location = df.job_location.fillna('NULL')

There are a couple of duplicates:  
Vilnius, Vilnius City Municipality, Lithuania  
Vilnius, Vilnius city municipality, Lithuania

In [41]:
df.job_location = df.job_location.str.replace('Vilnius, Vilnius city municipality, Lithuania', 'Vilnius, Vilnius City Municipality, Lithuania')

In [42]:
job_locations = pd.Series(df.job_location.drop_duplicates(), name = 'location_name')

# remove null values
job_locations[job_locations == 'NULL'] = np.nan

In [43]:
job_locations = job_locations.dropna().sort_values().reset_index(drop = True)
job_locations.index += 1

In [44]:
job_locations.to_csv('./csv/locations.csv', index = False)

I will check if the 'search_location' values are contained either in 'job_country' or 'job_location'

In [45]:
for s in df.search_location.unique():
    if s not in countries.values:
        print(s)

New York, United States
Illinois, United States
Texas, United States
California, United States
Florida, United States


I will set the 'search_location' values which contain *United States* (State, United States) to just *United States*

In [46]:
df.loc[df.search_location.str.contains('United States'), 'search_location'] = 'United States'

### Unique 'job_schedule_type'

In [47]:
# df.job_schedule_type.unique()

In [48]:
df.job_schedule_type = df.job_schedule_type.fillna('NULL')

Some job postings list multiple schedule types (e.g., 'Full-time, Part-time') as a single string. These strings need to be converted into lists of individual schedule types. This allows us to 'explode' the DataFrame later, creating a separate row for each schedule type associated with a single job posting, which is a key step in normalizing the data.

In [49]:
def convert_schedule_into_list(row):
    if row != 'NULL':
        row = row.replace(' and ', ',').replace(',,', ',').split(',')
        
    # for string with only one schedule,
    # convert to a list of only one element
    if isinstance(row, str):
        return [row]
    
    return row

In [50]:
df.job_schedule_type = df.job_schedule_type.apply(convert_schedule_into_list)

Now I want to get the list of unique 'job_schedule_type' values.

In [51]:
# create a list with all the individual values for job_schedule_type
all_schedules = []
for i in df.job_schedule_type.dropna():
    all_schedules += i

# clean the list
all_schedules = [i.strip() for i in all_schedules if i != 'NULL' ]

In [52]:
df.loc[df.job_schedule_type == 'NULL', 'job_schedule_type'] = np.nan

In [53]:
job_schedules = (pd.Series(all_schedules,name='schedule_type')
                 .drop_duplicates()
                 .sort_values()
                 .reset_index(drop = True)
)
job_schedules.index += 1

In [54]:
job_schedules

1    Contractor
2     Full-time
3    Internship
4     Part-time
5      Per diem
6     Temp work
7     Volunteer
Name: schedule_type, dtype: object

In [55]:
job_schedules.to_csv('./csv/job_schedules.csv', index = False)

### Unique 'job_via'

In [56]:
df.job_via = df.job_via.fillna('NULL')

In [57]:
df.job_via = df.job_via.str.replace('via ', '').str.strip()

Check the values of 'job_via' so I can find a pattern to clean the data.

In [58]:
# lowercase_job_via = df.job_via.str.lower().drop_duplicates().sort_values(key = lambda x : -x.str.len())

In [59]:
# print(*lowercase_job_via, sep='\n')

In [60]:
# print(*df.job_via.str.lower().drop_duplicates().sort_values(), sep='\n')

Let's check if there are variations of the main job portals:

In [61]:
# famous_job_portals = ['ZipRecruiter', 'Indeed', 'LinkedIn', 'Snagajob', 'Ai-Jobs.net', 
#         'Ladders', 'Dice', 'jobServe', 'Upwork', 'BeBee', 
#         'Built In', 'ProActuary', 'Remote OK', 'Get.It']
# famous_job_portals = [i.lower() for i in famous_job_portals]

In [62]:
# for i in famous_job_portals:
#     for j in df.job_via.str.lower().unique():
#         if i in j:
#             print(j)

There are some variations of the job portals 'BeBee', 'Indeed', 'JobServe' and 'Built In'

In [63]:
replacements = {
    'geebo' : 'geebo.com',
    'talentify' : 'Talentify',
    'tarta.ai' : 'tarta.ai',
    'linkedin' : 'LinkedIn',
    'bebee' : 'BeBee',
    'jobserve' : 'jobServer',
    'indeed' : 'Indeed',
    'built in' : 'Built In',
    'dice' : 'dice.com'
    }

for key, value in replacements.items():
    df.loc[df.job_via.str.lower().str.contains(key), 'job_via'] = value

In [64]:
df.loc[df.job_via.str.lower().str.contains('informs.org'), 'job_via'] = 'informs.org'
df.job_via = df.job_via.str.replace('Www.', 'www.')

In [65]:
df.job_via[df.job_via.str.endswith('-')]

24345    InternsVilla | Hub Of Internships -
Name: job_via, dtype: object

In [66]:
df.loc[df.job_via == 'InternsVilla | Hub Of Internships -', 'job_via'] = 'InternsVilla | Hub Of Internships'

Create the list of unique 'job_via' values

In [67]:
job_via = pd.Series(df.job_via.drop_duplicates(), name = 'job_via')
job_via[job_via == 'NULL'] = np.nan

job_via = job_via.dropna().sort_values().reset_index(drop = True)
job_via.index += 1

In [68]:
job_via.to_csv('./csv/job_via.csv', index = False)

### Unique 'job_skills'

In [69]:
df.job_skills.isna().sum()

np.int64(3187)

In [70]:
all_skills = df.job_skills.to_list()

all_skills = [i.replace('[', '')
              .replace(']', '')
              .replace("'", '')
              for i in all_skills if not isinstance(i, float)]

all_skills = ', '.join(all_skills)
all_skills = all_skills.split(', ')

job_skills = pd.Series(all_skills, name = 'skill')
job_skills = job_skills.drop_duplicates().sort_values().reset_index( drop = True)
job_skills.index += 1

In [71]:
job_skills

1         airflow
2        airtable
3         alteryx
4         angular
5      angular.js
          ...    
217     workfront
218         wrike
219       xamarin
220          yarn
221          zoom
Name: skill, Length: 221, dtype: object

In [72]:
job_skills.to_csv('./csv/skills.csv', index = False)

Explode the dataset to create a separate row for each skill listed in a job posting.

In [73]:
def expand_skills(row):
    if not isinstance(row, float): 
        skills_list = (row.replace('[', '')
                    .replace(']', '')
                    .replace("'", '')
        )
        skills_list = skills_list.split(', ')
        return skills_list
    else:
        return row

In [74]:
df.job_skills = df.job_skills.apply(expand_skills)

### Explode the dataset by `job_skills` and `job_schedule_type`

In [75]:
df2 = df.explode('job_schedule_type')
df2 = df2.explode('job_skills')

## Normalizing the Data

After creating the dimension tables (as Series with unique values), it's time to normalize the main DataFrame.

This involves replacing the text values in certain columns with their corresponding integer IDs from the dimension Series we created earlier.
 
The following columns will be normalized by mapping their string values to integer IDs:
- `job_title_short` -> `job_titles`
- `job_country` -> `countries`
- `job_location` -> `job_locations`
- `job_via` -> `job_via`
- `job_schedule_type` -> `job_schedules`
- `search_location` -> `countries`
- `company_name` -> `companies`
- `job_skills` -> `job_skills`

In [76]:
def normalize_col(series, column):
    s = pd.Series(series.index, index = series.values)
    df2[column] = df2[column].map(s)

In [77]:
unique_columns = {
    'job_title_short' : job_titles,
    'job_country' : countries,
    'job_location' : job_locations,
    'job_via' : job_via,
    'job_schedule_type' : job_schedules,
    'search_location' : countries,
    'company_name' : companies,
    'job_skills' : job_skills,
}

In [78]:
for key, value in unique_columns.items():
    normalize_col(value, key)

In [79]:
df2.sample(3)

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills
6943,9,Senior Analytics Engineer,2374.0,163.0,2.0,False,20,2023-01-11 18:17:04,False,False,20,year,157000.0,NaN,4859,18.0
15653,5,Data Scientist,86.0,111.0,2.0,True,107,2023-04-12 17:20:06,False,True,107,year,95000.0,NaN,6084,140.0
27288,3,Data Analyst,1325.0,163.0,2.0,False,107,2023-10-03 16:04:32,False,False,107,hour,NaN,27.5,4061,177.0


Sort the dataset by `job_posted_date` column.

In [80]:
df2 = df2.sort_values('job_posted_date').reset_index(drop = True)

### Renaming Columns for Database Schema Alignment

This step renames the columns in the DataFrame to match the column names defined in the SQL database schema. This ensures a smooth data loading process.

In [81]:
df2.rename(columns = {
    'job_title_short' : 'job_title_id',
    'job_title' : 'job_title_full',
    'job_location' : 'job_location_id',
    'job_via' : 'job_via_id',
    'job_schedule_type' : 'schedule_id',
    'search_location' : 'search_location_id',
    'job_location' : 'job_location_id',
    'job_work_from_home' : 'work_from_home',
    'job_no_degree_mention' : 'no_degree_mention',
    'job_health_insurance' : 'health_insurance',
    'job_country' : 'job_country_id',
    'company_name' : 'company_id',
    'job_skills' : 'skill_id'
}, inplace = True)

In [82]:
df2.to_csv('./csv/job_postings.csv', index = False)